In [22]:
# Import the required libraries:
import warnings
warnings.filterwarnings("ignore")

from utils import *

In [2]:
data = pd.read_csv("hmeq.csv")
data.head(5)

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,1,1100,25860.0,39025.0,HomeImp,Other,10.5,0.0,0.0,94.366667,1.0,9.0,NaN
1,1,1300,70053.0,68400.0,HomeImp,Other,7.0,0.0,2.0,121.833333,0.0,14.0,NaN
2,1,1500,13500.0,16700.0,HomeImp,Other,4.0,0.0,0.0,149.466667,1.0,10.0,NaN
3,1,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1700,97800.0,112000.0,HomeImp,Office,3.0,0.0,0.0,93.333333,0.0,14.0,NaN


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5960 entries, 0 to 5959
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   BAD      5960 non-null   int64  
 1   LOAN     5960 non-null   int64  
 2   MORTDUE  5442 non-null   float64
 3   VALUE    5848 non-null   float64
 4   REASON   5708 non-null   object 
 5   JOB      5681 non-null   object 
 6   YOJ      5445 non-null   float64
 7   DEROG    5252 non-null   float64
 8   DELINQ   5380 non-null   float64
 9   CLAGE    5652 non-null   float64
 10  NINQ     5450 non-null   float64
 11  CLNO     5738 non-null   float64
 12  DEBTINC  4693 non-null   float64
dtypes: float64(9), int64(2), object(2)
memory usage: 605.4+ KB


###  **Données manquantes**


La méthodologie employée pour imputer les données manquantes est la suivante : 

RQ : On considère qu'au delà de 40% de valeurs manquantes, une stratégie d'imputation risque d'introduire un biais dans l'analyse donc on va pas utiliser une telle variable par la suite.

- Variables quantitatives : 
On va se baser sur la distribution de la variable, si sa distribution est asymétrique, on va imputer par la médiane et sinon par la moyenne.
- Variables qualitatives : 
On va considérer le % de données manquantes, si celui-ci est inférieur à 15%, on va imputer par la valeur la plus fréquente (donc le mode), sinon on sera amené à créer une nouvelle classe nommée "autres". 

Les valeurs imputées dans le train_set seront repercutées dans le test_set

In [48]:
train, test = train_test_split(data, test_size = 0.2, random_state = 42)

In [49]:
print("data shape", data.shape)
print("test shape", test.shape)
print("train shape", train.shape)


data shape (5960, 13)
test shape (1192, 13)
train shape (4768, 13)


In [39]:
train.columns

Index(['BAD', 'LOAN', 'MORTDUE', 'VALUE', 'REASON', 'JOB', 'YOJ', 'DEROG',
       'DELINQ', 'CLAGE', 'NINQ', 'CLNO', 'DEBTINC'],
      dtype='object')

In [44]:
#prct_num, prct_qual = utils.prct_nan(train)

In [50]:
vars_num = [train.select_dtypes('float').columns]
vars_qual = [train.select_dtypes('object').columns]

In [41]:
#prct_num = [print((100*train[col].isna().sum()/train.shape[0]).sort_values(ascending = False)) for col in vars_num]
#prct_qual = [print((100*train[col].isna().sum()/train.shape[0]).sort_values(ascending = False)) for col in vars_qual]

In [51]:
prct_num, prct_qual = utils.prct_nan(train)

Pour les variables numériques, on a : 

DEBTINC    20.931208
DEROG      11.577181
DELINQ      9.584732
MORTDUE     8.640940
YOJ         8.473154
NINQ        8.389262
CLAGE       5.012584
CLNO        3.544463
VALUE       1.614933
dtype: float64
-----
Pour les variables quantitatives, on a : 

JOB       4.656040
REASON    4.089765
dtype: float64


In [52]:
prct_num, prct_qual = utils.prct_nan(test)

Pour les variables numériques, on a : 

DEBTINC    22.567114
DEROG      13.087248
DELINQ     10.318792
YOJ         9.312081
NINQ        9.228188
MORTDUE     8.892617
CLAGE       5.788591
CLNO        4.446309
VALUE       2.936242
dtype: float64
-----
Pour les variables quantitatives, on a : 

REASON    4.781879
JOB       4.781879
dtype: float64


In [53]:
train, test = utils.imputation(train, test)

AVANT IMPUTATION


**Variables quantitatives**:

---Train set----
Colonnes asymétriques dans le train ['MORTDUE', 'VALUE', 'DEROG', 'DELINQ', 'CLAGE', 'NINQ', 'DEBTINC']
Colonnes symétriques dans le train ['YOJ', 'CLNO']
----Test set----
Colonnes asymétriques dans le test ['MORTDUE', 'VALUE', 'YOJ', 'DEROG', 'DELINQ', 'NINQ', 'DEBTINC']
Colonnes symétriques dans le test ['CLAGE', 'CLNO']


**Variables qualitatives**:

---Train set----
Colonnes avec - de 15% de nan dans le train ['REASON', 'JOB']
Colonnes avec + de 15% de nan dans le train []
----Test set----
Colonnes avec - de 15% de nan dans le test ['REASON', 'JOB']
Colonnes avec + de 15% de nan dans le test []



APRÉS IMPUTATION
Valeurs manquantes dans le test set

BAD        0
LOAN       0
MORTDUE    0
VALUE      0
REASON     0
JOB        0
YOJ        0
DEROG      0
DELINQ     0
CLAGE      0
NINQ       0
CLNO       0
DEBTINC    0
dtype: int64


Valeurs manquantes dans le train set

BAD        0
LOAN       0
MORTDUE    0
VALUE     

In [54]:
data_clean = pd.concat([train.reset_index(drop=True), test.reset_index(drop=True)], axis= 0)

In [14]:
data_clean.head()

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,0,9100,62976.0,77829.0,HomeImp,Mgr,0.000000,0.0,2.0,131.193496,0.0,23.0,26.090419
1,0,43800,5884.0,64890.0,HomeImp,Office,8.938187,0.0,0.0,309.827548,0.0,19.0,16.160451
2,0,13800,75142.0,95667.0,DebtCon,Mgr,19.000000,0.0,0.0,230.457554,10.0,24.0,33.383314
3,1,2000,45000.0,55000.0,HomeImp,Other,3.000000,0.0,0.0,86.066667,2.0,25.0,34.738063
4,0,9900,47367.0,114837.0,DebtCon,ProfExe,4.000000,0.0,0.0,210.435534,0.0,10.0,25.042109


In [20]:
data_clean.shape

(5960, 13)